In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import coronavirus_report as corona

In [ ]:
SAVE_DATA_PATH         = './data'
SAVE_WORLD_PLOT_PATH   = './plots/world'
SAVE_COUNTRY_PLOT_PATH = './plots/countries'

#COUNTRIES = ['Spain', 'Portugal']
#COUNTRIES = ["Italy", "US", "Spain", "France"]
COUNTRIES = ["Colombia", "Argentina", "Mexico", "Peru", "Chile", "Brazil", "Ecuador", "Uruguay", "Paraguay", "Venezuela", "Bolivia"] 
group_name = 'LATAM'

In [ ]:
corona.create_folder(folder_path = SAVE_DATA_PATH)
corona.create_folder(folder_path = SAVE_WORLD_PLOT_PATH)
corona.create_folder(folder_path = SAVE_COUNTRY_PLOT_PATH + "/" + group_name)

# Download Data from Github Repositoty

In [ ]:
df_confirmed, df_deaths, df_mortality = corona.get_data(savePath = SAVE_DATA_PATH)
df_confirmed.tail()

# DYNAMIC EVOLUTION

In [ ]:
variable = 'Mortality'

if variable == "Cases" or variable == "C":
    data = df_confirmed
    var_name = 'Cases Confirmed'
elif variable == "Deaths" or variable == "D":
    data = df_deaths
    var_name = 'Deaths Confirmed'
elif variable == "Mortality" or variable == "M":
    data = df_mortality
    var_name = f'Mortality Rate (More than 100 cases)'

corona.dynamic_evolution(data, var_name = var_name, number_of_countries = 25) 

# WORD

In [ ]:
# Create confirmed cases and deaths world series Including out China
df_confirmed_word = df_confirmed.sum(axis=1)
df_confirmed_word.columns = ["World"]
df_deaths_word = df_deaths.sum(axis=1)
df_deaths_word.columns = ["World"]

# Create confirmed cases world series with out China
df_confirmed_word_noChina = df_confirmed.loc[:,df_confirmed.columns != 'China'].sum(axis=1)
df_confirmed_word_noChina.columns = ["Word - No China"]

In [ ]:
corona.log_scale_plot(df_confirmed_word_noChina, "Cases Word No China", save_path = f"{SAVE_WORLD_PLOT_PATH}/log_world_cases.png")

### Exponential Factor $R^2$

In [ ]:
corona.exponential_r2(df_confirmed, threshold = 200)

In [ ]:
corona.make_ts_plot(df_confirmed_word, "World - Cases", save_path = f"{SAVE_WORLD_PLOT_PATH}/world_cases.png")
df_confirmed_word.tail()

In [ ]:
corona.make_ts_plot(df_deaths_word, "World - Deaths", save_path = f"{SAVE_WORLD_PLOT_PATH}/world_deaths.png")
df_deaths_word.tail()

In [ ]:
mortality = (df_deaths_word/df_confirmed_word) * 100
mortality.columns = ['Mortality']
corona.make_ts_plot(mortality, "World - Mortality", save_path = f"{SAVE_WORLD_PLOT_PATH}/world_mortality.png")

## Evolution of the factor cummulative

In [ ]:
corona.factor_evolution(df_confirmed_word , num_last_days = 60, start_d = 5, save_path = f"{SAVE_WORLD_PLOT_PATH}/world_factor_cum_evol.png")

## Evolution of the factor

In [ ]:
factor_smt_word = corona.moving_median(df_confirmed_word, window=7, last_n_days = 45, save_path = f"{SAVE_WORLD_PLOT_PATH}/world_factor_evol.png")

## Factor Change

In [ ]:
dataFactor = corona.factor_plot(df_confirmed, window = 7, var_name = 'Today', past_days_contrast = 7, save_path = f"{SAVE_WORLD_PLOT_PATH}/world_factor_today.png") # var_name = 'Change' or 'Today'
dataFactor.head()

In [ ]:
dataFactor = corona.factor_plot(df_confirmed, window = 7, var_name = 'Change', past_days_contrast = 7, save_path = f"{SAVE_WORLD_PLOT_PATH}/world_factor_change.png") # var_name = 'Change' or 'Today'

# BY COUNTRY

In [ ]:
data1 = df_confirmed.loc[:, COUNTRIES]
data2 = df_deaths.loc[:, COUNTRIES]

# Total Cases

In [ ]:
corona.make_ts_plot(data1.iloc[len(data1)-30:,:], "Total Cases", save_path = f"{SAVE_COUNTRY_PLOT_PATH}/{group_name}/{group_name}_cases.png") #data1.iloc[:len(data1)-1,:]
data1.tail()

# Factors

In [ ]:
for i in data1.columns:
    smt_f = corona.moving_median(data1.iloc[:len(data1)-1,:][i], window=7, last_n_days = 40, spec_name = i, show_quantiles = True, ylim_top = 1.45, plotIt = True, save_path = f"{SAVE_COUNTRY_PLOT_PATH}/{group_name}/{group_name}_factor_evol_{i}.png")

In [ ]:
dataFactor = corona.factor_plot(data1, window = 7, var_name = 'Today', past_days_contrast = 7, save_path = f"{SAVE_COUNTRY_PLOT_PATH}/{group_name}/{group_name}_factor_today.png") # var_name = 'Change' or 'Today'
dataFactor

# Daily Plot

In [ ]:
df, plot_daily = corona.daily_plot(data1.iloc[:len(data1)-1,:], num_days = 60, threshold_cases = 50, save_path = f"{SAVE_COUNTRY_PLOT_PATH}/{group_name}/{group_name}_dalily_cases.png")
# df

### Factor -> I(n+1)/I(n)

In [ ]:
# Factor by country of the last n days sorted by median.
d_factor, plt = corona.create_factor_stats(data1, last_n_days=7, plotIt=True, plt_qrt = ["50"], save_path = f"{SAVE_COUNTRY_PLOT_PATH}/{group_name}/{group_name}_factor_stats.png") #"75","50","25","min"
d_factor

# Forecast

In [ ]:
forecast_n    = 5  # Number of days to forecast
history       = 7 # History to calculate the factor
history_plot  = 30  # Number of days to plot before

forecast_low, forecast_med, forecast_hig = corona.forecast_corona(data1, # data1.iloc[:len(data1)-1,:]
                                                forecast_n = forecast_n, 
                                                history_plot = history_plot, 
                                                history = history,
                                                highlight_country = "",#data1.columns, #data1.columns,
                                                plot_intervals = False, 
                                                save_path = f"{SAVE_COUNTRY_PLOT_PATH}/{group_name}/{group_name}_forecast_cases.png")
forecast_med

In [ ]:
forecast_low

# DEATHS

In [ ]:
corona.make_ts_plot(data2.iloc[len(data1)-20:len(data1)-1,:], "Total Deaths", save_path = f"{SAVE_COUNTRY_PLOT_PATH}/{group_name}/{group_name}_deaths.png")
data2.tail()

In [ ]:
df, plot_daily = corona.daily_plot(data2.iloc[:len(data1)-1,:], num_days = 30, save_path = f"{SAVE_COUNTRY_PLOT_PATH}/{group_name}/{group_name}_daily_deaths.png")

In [ ]:
d_factor = corona.create_factor_stats(data2.iloc[:len(data1)-1,:], last_n_days=10, plotIt=True , plt_qrt = ["50"], save_path = f"{SAVE_COUNTRY_PLOT_PATH}/{group_name}/{group_name}_factor_stats_deaths.png")
d_factor

In [ ]:
forecast_n    = 5  # Number of days to forecast
history       = 10 # History to calculate the factor
history_plot  = 15  # Number of days to plot before

forecast_low, forecast_med, forecast_hig = corona.forecast_corona(data2.iloc[:len(data1)-1,:], # data2.iloc[:len(data1)-1,:]
                                                forecast_n = forecast_n, 
                                                history_plot = history_plot, 
                                                history = history,
                                                highlight_country = data1.columns, #data1.columns
                                                plot_intervals = False,
                                                decrease_factor_speed = 1, 
                                                save_path = f"{SAVE_COUNTRY_PLOT_PATH}/{group_name}/{group_name}_forecast_deaths.png") # 0.99
forecast_med